In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import datetime as dt


# Defining teams and codes

In [2]:
dict_longteam_to_shortteam = {'anaheim-ducks':'ANA','arizona-coyotes':'ARI','boston-bruins':'BOS','buffalo-sabres':'BUF',
                              'calgary-flames':'CAL','carolina-hurricanes':'CAR','chicago-blackhawks':'CHI','columbus-blue-jackets':'CLB',
                              'colorado-avalanche':'COL','dallas-stars':'DAL','detroit-red-wings':'DET','edmonton-oilers':'EDM',
                              'florida-panthers':'FLA','los-angeles-kings':'LA','minnesota-wild':'MIN','montreal-canadiens':'MON',
                              'nashville-predators':'NAS','new-jersey-devils':'NJ','new-york-islanders':'NYI','new-york-rangers':'NYR',
                              'ottawa-senators':'OTT','philadelphia-flyers':'PHI','pittsburgh-penguins':'PIT','san-jose-sharks':'SJ',
                              'st.-louis-blues':'STL','tampa-bay-lightning':'TB','toronto-maple-leafs':'TOR','vancouver-canucks':'VAN',
                              'vegas-golden-knights':'VEG','washington-capitals':'WAS','winnipeg-jets':'WIN',
                             }

dict_shortteam_to_codeteam = {'ANA':2001,'ARI':2002,'BOS':2003,'BUF':2004,'CAL':2005,'CAR':2006,
                              'CHI':2007,'CLB':2008,'COL':2009,'DAL':2010,'DET':2011,'EDM':2012,
                              'FLA':2013,'LA':2014,'MIN':2015,'MON':2016,'NAS':2017,'NJ':2018,
                              'NYI':2019,'NYR':2020,'OTT':2021,'PHI':2022,'PIT':2023,'SJ':2024,
                              'STL':2025,'TB':2026,'TOR':2027,'VAN':2028,'VEG':2029,'WAS':2030,
                              'WIN':2031,
                             }

dict_codeteam_to_shortteam = {2001:'ANA',2002:'ARI',2003:'BOS',2004:'BUF',2005:'CAL',2006:'CAR',
                              2007:'CHI',2008:'CLB',2009:'COL',2010:'DAL',2011:'DET',2012:'EDM',
                              2013:'FLA',2014:'LA',2015:'MIN',2016:'MON',2017:'NAS',2018:'NJ',
                              2019:'NYI',2020:'NYR',2021:'OTT',2022:'PHI',2023:'PIT',2024:'SJ',
                              2025:'STL',2026:'TB',2027:'TOR',2028:'VAN',2029:'VEG',2030:'WAS',
                              'WIN':2031,
                             }

# Reading actual historical data

In [3]:
teams = ['carolina-hurricanes','chicago-blackhawks','columbus-blue-jackets','dallas-stars'
        ,'detroit-red-wings','florida-panthers','nashville-predators','tampa-bay-lightning'
        ,'boston-bruins','buffalo-sabres','new-jersey-devils','new-york-islanders'
        ,'new-york-rangers','philadelphia-flyers','pittsburgh-penguins','washington-capitals'
        ,'calgary-flames','edmonton-oilers','montreal-canadiens','ottawa-senators'
        ,'toronto-maple-leafs','vancouver-canucks','winnipeg-jets'
        ,'anaheim-ducks','arizona-coyotes','colorado-avalanche','los-angeles-kings'
        ,'minnesota-wild','san-jose-sharks','st.-louis-blues','vegas-golden-knights']
# teams = ['vegas-golden-knights']
# seasons = ['2006-2007','2007-2008','2008-2009','2009-2010','2010-2011','2011-2012'
#            ,'2012-2013','2013-2014','2014-2015','2015-2016','2016-2017','2017-2018','2018-2019','2019-2020',]
seasons = ['2020-2021']

team_season_incorret = []

my_columns = ['Season','Team','Date', 'VS', 'Score', 'ML', 'O/U', 'Goalie', 'Opp. Goalie']
df = pd.DataFrame(columns = my_columns)
for team in tqdm(teams):
    
    for season in seasons:
        try:
            r = pd.read_html(f'https://www.covers.com/sport/hockey/nhl/teams/main/{team}/{season}', match='Regular Season')
            r[0].insert(0,'Season',int)
            r[0].insert(1,'Team',int)
            r[0]['Season'] = season
            r[0]['Team'] = team
            r[0].columns = my_columns
            df = df.append(r[0])
        except:
            team_season_incorret.append((team,season))

# df.drop(['filled-1', 'filled-2'], axis=1, inplace=True)

# Extracting the year to attach to the field Date
year_season_star =df['Season'].str.split('-').str[0]
year_season_end =df['Season'].str.split('-').str[1]

# Attaching the year to the field Date
df['Date'] = list(np.select([df['Date'].str[:3].isin(['Oct','Nov','Dec']), 
                             ~df['Date'].str[:3].isin(['Oct','Nov','Dec'])] , 
                            [df['Date']+'-'+year_season_star, df['Date']+'-'+year_season_end], default= 0))

df['Date'] = pd.to_datetime(df['Date'],)
df = df.iloc[::-1]
df['GameCounter'] = df.groupby(['Team','Season']).cumcount()+1
df.set_index('GameCounter', inplace=True)



100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:30<00:00,  1.02it/s]


In [4]:
# Changing long name in team for the short name; Creating new column codeTeam and assigning the code
df['Team'] = df['Team'].replace(to_replace=dict_longteam_to_shortteam, value=None)
df['VS'] = df['VS'].str.replace('NEUTRAL','')

# Saving the data in csv

In [6]:
df.to_csv('Sports_NHL_History.csv')